In [ ]:
!pip install taichi

     |████████████████████████████████| 27.2 MB 1.5 MB/s 


丶


In [ ]:
import taichi as ti


[Taichi] version 0.9.1, llvm 10.0.0, commit e2e0e669, linux, python 3.7.12


In [ ]:
# ti.init(arch=ti.cuda)
ti.init(arch=ti.cpu)

[Taichi] Starting on arch=x64


In [ ]:
N = 128
x = ti.Vector.field(3, float, (N, N))
y = ti.Vector.field(3, float, (N, N))

这三行声明 x 和 v 为大小为 n 乘以 n 的2d 数组，其中数组的每个元素都是浮点数的三维向量。在太极拳中，阵列被称为“场”，这两个场分别记录点质量的位置和速度。注意，如果你在 CUDA GPU 上初始化了太极，这些字段/数组将自动存储在 GPU 内存中。

In [ ]:
ball_radius = 0.2
ball_center = ti.Vector.field(3, float, (1,))

这里，球心是一个大小为1的一维场，它的单个分量是一个三维的浮点向量。

在声明了所需的字段之后，让我们使用相应的数据在 t = 0处初始化这些字段。我们希望确保对于同一行或列上的任何一对相邻点，它们之间的距离等于单元格 _ size = 1.0 / N。下面的初始化例程确保了这一点:

In [ ]:
cell_size = 1.0 / N

def init_scene():
  for i, j in ti.ndrange(N, N):
    x[i, j] = ti.Vector([i * cell_size,
                                  j * cell_size / ti.sqrt(2),
                                  (N - j ) * cell_size / ti.sqrt(2)])
    print([i, j, (N - j)])
    print([i * cell_size,  j * cell_size / ti.sqrt(2), (N - j ) * cell_size / ti.sqrt(2)])
    ball_center[0] = ti.Vector([0.5, -0.5, 0.0])
init_scene()

Streaming output truncated to the last 5000 lines.
[97, 82, 46]
[0.7578125, 0.452990281697632, 0.2541164994889155]
[97, 83, 45]
[0.7578125, 0.4585145534256519, 0.2485922277608956]
[97, 84, 44]
[0.7578125, 0.4640388251536718, 0.2430679560328757]
[97, 85, 43]
[0.7578125, 0.4695630968816917, 0.23754368430485578]
[97, 86, 42]
[0.7578125, 0.47508736860971157, 0.2320194125768359]
[97, 87, 41]
[0.7578125, 0.48061164033773146, 0.226495140848816]
[97, 88, 40]
[0.7578125, 0.4861359120657514, 0.22097086912079608]
[97, 89, 39]
[0.7578125, 0.4916601837937713, 0.2154465973927762]
[97, 90, 38]
[0.7578125, 0.4971844555217912, 0.20992232566475627]
[97, 91, 37]
[0.7578125, 0.5027087272498111, 0.20439805393673638]
[97, 92, 36]
[0.7578125, 0.508232998977831, 0.1988737822087165]
[97, 93, 35]
[0.7578125, 0.5137572707058509, 0.19334951048069657]
[97, 94, 34]
[0.7578125, 0.5192815424338708, 0.18782523875267668]
[97, 95, 33]
[0.7578125, 0.5248058141618908, 0.18230096702465678]
[97, 96, 32]
[0.7578125, 0.530330

KeyboardInterrupt: ignored

上面的内容还是笔记费解的，

## 模拟

一堆点就是一个社会，每个热都受到一堆力，有力才会运动

In [ ]:
@ti.kernel
def step():
  for i in ti.ground(v):
    v[i].y -= gravity * dt

In [ ]:
links = [[-1, 0], [1, 0], [0, -1], [0, 1], [-1, -1], [1, -1], [-1, 1], [1, 1]]
links = [ti.Vector(v) for v in links]

In [ ]:
for i in ti.grouped(x):
  force = ti.Vector([0.0, 0.0, 0.0])
  for d in ti.static(links):
    j = min(max(i + d, 0), [N-1, N-1])
    relative_pos = x[j] - x[i]
    current_length = relative_pos.norm()
    original_length = cell_size * float(i-j).norm()
    if 

In [ ]:
for i in ti.grounped(x):
  v[i] *= ti.exp(-damping * dt)

In [ ]:
if (x[i] - ball_center[0]).norm() <= ball_radius: # x[i]表示布料质点
  v[i] = ti.Vector([0.0, 0.0, 0.0])

In [ ]:
x[i] += dt * v[i] # 这是向量乘法吧？x[i]是空间三维的坐标